## This demo focuses on the following two stories:
- RSPY-327
- RSPY-397

The goal of these stores was to identify which assets from an item already exist in the S3 catalog bucket and transfer only the new assets from the temporary S3 bucket to the final S3 bucket. When updating an item using the PUT request method, any asset present in the catalog but not included in the request is removed from the catalog and physically deleted from the S3 bucket. Additionally, the code needed to be updated (for both rs-server-catalog and rs-client-libraries) to support using the filename as the key in the `Asset` section.
```
"assets": {
        "first-file.EOF": {
            "href": "https://dev-rspy.esa-copernicus.eu/catalog/collections/user:S1A_aux/items/my-item/download/first-file.EOF",
            "alternate": {
                "s3": {
                    "href": "s3://rs-cluster-catalog/user/S1A_aux/first-file.EOF"
                }
            },
        "second-file.EOF": {
            "href": "https://dev-rspy.esa-copernicus.eu/catalog/collections/user:S1A_aux/items/my-item/download/second-file.EOF",
            "alternate": {
                "s3": {
                    "href": "s3://rs-cluster-catalog/user/S1A_aux/second-file.EOF"
                }
            },
        ......
        }
```

In [ ]:
# Init environment before running a demo notebook.
from resources.utils import *
user = "jovyan" if not local_mode else "user"
init_demo(owner_id = user)
from resources.utils import * # reload the global vars again

## Prepare the enviornment
A test collection needs to be created for this demo. Additionally, 4 chunks from the CADIP station will be staged in the temporary S3 bucket. By using these files, 4 assets will be also created

In [ ]:
# Create a test collection 
collection = create_test_collection()
# Download 4 files from the cadip station
files = cadip_client.search_stations(start_date, stop_date, limit=4)
assets:list[dict] = []
for count, file in enumerate(files):
        first_filename = file["id"]
        s3_path = (
            f"s3://{RSPY_TEMP_BUCKET}/{cadip_client.apikey_user_login}/{cadip_client.station_name}"
        )
        temp_s3_file = f"{s3_path}/{first_filename}"        
        local_path = None
        # Call the staging service
        cadip_client.staging(first_filename, s3_path=s3_path, tmp_download_path=local_path)
        # Then we can check when the staging has finished by calling the check status service
        while True:
            status = cadip_client.staging_status(first_filename)
            print(f"Staging status for {first_filename!r}: {status.value}")
            if status in [EDownloadStatus.DONE, EDownloadStatus.FAILED]:
                print("\n")
                break
            sleep(1)
        assert status == EDownloadStatus.DONE, "Staging has failed"

        # The filename from the temp s3 bucket is set as the key in the dictionary
        assets.insert(0, {temp_s3_file.split("/")[-1] : Asset(href = temp_s3_file)})
            
import pprint
pprint.PrettyPrinter(indent=4).pprint(assets)

## Define simulated values

In [ ]:
HEADERS={"x-api-key": apikey}

REQUEST_TIMEOUT = 5
ITEM_NAME = "CADIP_SESSION_ITEM_DEMO"

# Simulated values
WIDTH=2500
HEIGHT=2500
# Create a catalog item with one asset, the first one from the list of cadip files we downloaded
geometry = {
    "type": "Polygon",
    "coordinates": [[[-180, -90], [180, -90], [180, 90], [-180, 90], [-180, -90]]],
}
bbox = [-180.0, -90.0, 180.0, 90.0]

properties = {
    "gsd": 0.12345,
    "width": WIDTH,
    "height": HEIGHT,
    "datetime": datetime.now(),
    "proj:epsg": 3857,
    "orientation": "nadir",
}

## Create an item
The item will include only one asset, using the first asset from the assets list we previously created 

In [ ]:
import requests
import json

item = Item(
    collection=collection.id,
    id=ITEM_NAME,
    geometry=geometry,
    bbox=bbox,
    datetime=datetime.now(),
    properties=properties,
    assets=assets[0])

pprint.PrettyPrinter(indent=4).pprint(item.to_dict())



# Send the POST request
This action has the role to initially publish the item in the catalog

In [ ]:
# Sending the post request to publish the item
response = requests.post(
    f"{stac_client.href_catalog}/catalog/collections/{collection.id}/items/",
    json=item.to_dict(),
    headers=HEADERS,
    timeout = REQUEST_TIMEOUT,
)
response.raise_for_status()

# Reading the newly inserted item from the catalog
print("Reading the item\n")
filter_ = {
        "op": "and",
        "args": [
            {"op": "=", "args": [{"property": "collection"}, f"{user}_{TEST_COLLECTION}"]},
            {"op": "=", "args": [{"property": "owner"}, user]},
            {"op": "=", "args": [{"property": "id"}, ITEM_NAME]},
        ],
    }

search = stac_client.search(filter=filter_)
assert len(list(search.items_as_dicts())) == 1
catalog_item = list(search.items_as_dicts())[0]
pprint.PrettyPrinter(indent=4).pprint(catalog_item)

## Add a new asset to the already published item. 

In [ ]:
# Add a new asset to the already published item
catalog_item["assets"][list(assets[1].keys())[0]] = list(assets[1].values())[0].to_dict()
pprint.PrettyPrinter(indent=4).pprint(catalog_item)


## Send the PUT request to update the item from the catalog
The `rs-server-catalog` should transfer only the new asset from s3://rs-temp-bucket to s3://rs-catalog-bucket. It should ignore any asset that already exists both in the catalog and physically in s3://rs-catalog-bucket. To check this behavior, we have to look in the `rs-server-catalog` logs.

In [ ]:
response = requests.put(
    f"{stac_client.href_catalog}/catalog/collections/{TEST_COLLECTION}/items/{catalog_item['id']}",
    json=catalog_item,
    **apikey_headers,
    timeout = REQUEST_TIMEOUT,
)
pprint.PrettyPrinter(indent=4).pprint(json.loads(response.content))

## Attempt to change the path of the newly inserted asset.
An error message should come from the `rs-server-catalog`

In [ ]:
from starlette.status import ( 
    HTTP_400_BAD_REQUEST,    
)
# Setting another path for the previously inserted asset
print("Trying to change the path of the previoulsy inserted asset")
catalog_item["assets"][list(assets[1].keys())[0]] = {"href": f"s3://rs-cluster-temp/another/path/to/{list(assets[1].keys())[0]}"}
response = requests.put(
    f"{stac_client.href_catalog}/catalog/collections/{TEST_COLLECTION}/items/{catalog_item['id']}",
    json=catalog_item,
    **apikey_headers,
    timeout = REQUEST_TIMEOUT,
)
assert response.status_code == HTTP_400_BAD_REQUEST
print("The response from rs-server-catalog:\n")
print(json.loads(response.content))

## Retrieve the item from the catalog again to ensure synchronization.

In [ ]:
search = stac_client.search(filter=filter_)
assert len(list(search.items_as_dicts())) == 1
catalog_item = list(search.items_as_dicts())[0]
pprint.PrettyPrinter(indent=4).pprint(catalog_item)

## Add two new assets to the item but exclude the first one

In [ ]:
# Delete the first item from the item
catalog_item["assets"].pop(next(iter(catalog_item["assets"])))
# Add two new assets to the item (the last two chunks from the assets list we created at the beginning)
for asset in assets[2:]:
    catalog_item["assets"][list(asset.keys())[0]] = list(asset.values())[0].to_dict()
pprint.PrettyPrinter(indent=4).pprint(catalog_item)

## Send the PUT request to update the item
This request will update the item in the catalog by transferring the two newly added CADIP chunks from the temporary S3 catalog to the final S3 catalog. The non-existing asset will be removed from the catalog as well as physically from the catalog s3 bucket. This behavior can be observed by checking the `rs-server-catalog` logs.

In [ ]:
response = requests.put(
    f"{stac_client.href_catalog}/catalog/collections/{TEST_COLLECTION}/items/{catalog_item['id']}",
    json=catalog_item,
    **apikey_headers,
    timeout = REQUEST_TIMEOUT,
)
response.raise_for_status() 
catalog_item = json.loads(response.content)
pprint.PrettyPrinter(indent=4).pprint(catalog_item)